# NP Expression and Co-Expression Mapper

In [ ]:
# Important to keep this in sync whenever renaming this notebook
notebook_name = 'NP_Scope'

# Import modules
import pymap as pm # SJS module: pymap.py must reside in current directory
import json
import shutil
import psutil
import os
import zipfile
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.colors as mcol
import matplotlib.colors as mcolors
import matplotlib.patches as patches
import seaborn as sns
from matplotlib.patches import Rectangle, Polygon
from matplotlib.backends.backend_pdf import PdfPages

# Make some choices...
#dataset_list = ['Y21ss'] #,'Y21tenX','Y23']
#dataset = 'Y21ss'
dataset = 'Y23'
min_tally = 10 if dataset == 'Y21ss' else 100
threshold = 10 if dataset == 'Y21ss' else 1
sns.set_theme(font_scale=1.3, rc={'patch.linewidth': 0})

# Assemble the general filepaths
name_root = notebook_name
results_path = 'Products/'+name_root+'/'
pdf_path = results_path+'PDFs/'
csv_path = results_path+'CSVs/'
zip_path = results_path+'ZIPs/'
temp_path = 'Temp/'

#for dataset in dataset_list:

# Read annotated expression data
source_path = 'Resources/'+dataset+'_NP_counts.csv.zip' # 'NP' or 'nMod'
support_path = 'Resources/'+dataset+'_NP_support.json' # 'NP' or 'nMod'
print('Reading',source_path)
df = pd.read_csv(source_path)
anno_counts = df.copy()
# Map expression in selected cell types only
#anno_counts = df[df['type'] == '01 Neuronal'].copy()
#anno_counts = df[df['type'] == '02 Glial'].copy()

# Read support dictionary
with open(support_path, 'r') as fp:
    support_dict = json.load(fp)

# Extract annotation value dictionaries
region_colors = dict(zip(anno_counts['region'],anno_counts['region_color']))
class_colors = dict(zip(anno_counts['class'],anno_counts['class_color']))
class_colors = {key: class_colors[key] for key in sorted(class_colors)}
subclass_colors = dict(zip(anno_counts['subclass'],anno_counts['subclass_color']))
subclass_colors = {key: subclass_colors[key] for key in sorted(subclass_colors)}
subclass_zones = dict(zip(anno_counts['subclass'],anno_counts['class']))
subclass_zones = {key: subclass_zones[key] for key in sorted(subclass_zones)}
subclass_shorts = dict(zip(anno_counts['subclass'],anno_counts['subclass']))
subclass_shorts = {key: subclass_shorts[key].split(' ',1)[1] for key in sorted(subclass_shorts)}
cluster_colors = dict(zip(anno_counts['cluster'],anno_counts['cluster_color']))
cluster_colors = {key: cluster_colors[key] for key in sorted(cluster_colors)}
cluster_zones = dict(zip(anno_counts['cluster'],[s.split(' ',1)[1] for s in anno_counts['cluster']]))
cluster_zones = {key: cluster_zones[key] for key in sorted(cluster_zones)}

# Read CSV table to form region dictionaries
df = pd.read_csv('Resources/'+dataset+'_region_table.csv')
region_shorts = df.set_index('region_label')['region_short'].to_dict()
region_zones = df.set_index('region_label')['zone'].to_dict()

# Form annotation unique value lists
types = (list(set(anno_counts['type']))); types.sort()
regions = list(region_colors.keys())
classes = list(class_colors.keys())
subclasses = list(subclass_colors.keys())
clusters = list(cluster_colors.keys())

if dataset == 'Y23': # 'supertypes' in Y23 dataset only
    supertype_colors = dict(zip(anno_counts['supertype'],anno_counts['supertype_color']))
    supertype_colors = {key: supertype_colors[key] for key in sorted(supertype_colors)}
    supertypes = list(supertype_colors.keys())

if dataset[0:3] == "Y21": # 'neighborhood' in Y21 datasets only
    neighborhood_colors = dict(zip(anno_counts['neighborhood'],anno_counts['neighborhood_color']))
    neighborhood_colors = {key: neighborhood_colors[key] for key in sorted(neighborhood_colors)}
    neighborhoods = list(neighborhood_colors.keys())

# Read CSV table to form gene label dictionary
df = pd.read_csv('Resources/All_nMods_labeled.csv', usecols=[1,2], index_col=0)
gene_labels = df.to_dict()['Label']

# Configure multiindex
major_annos = ['type','region','class','subclass','cluster']
all_annos = support_dict['annotation_labels']
minor_annos = [s for s in all_annos if s not in major_annos]
anno_counts.drop(columns=minor_annos)
anno_counts.set_index(major_annos, inplace=True)

# Sanity check
print('Sanity check:\n'+str(anno_counts[['Cck','Npy','Vip','Cckbr','Kng1','Bdkrb1']].sum()))

# Use conjugate pair table to generate aggregate NP_GPCRs conjugate to a set of NPPs

# Read pair table
pair_table = pd.read_csv('Resources/Mouse_NP_pairs.csv')
pair_table['Pair Label'] = pair_table['NPP'] + '->' + pair_table['NP-GPCR']
pair_id = list(pair_table['Pair ID'])
pair_npps = list(pair_table['NPP'])
pair_nprs = list(pair_table['NP-GPCR'])
pair_labels = list(pair_table['Pair Label'])
pair_table.set_index('Pair ID', inplace=True)

# Build cognate NPP/aggregate-NP-GPCRs dictionary
df = pair_table[['NPP','NP-GPCR']].set_index('NPP').copy()
df['Aggs'] = df['NP-GPCR']+' '
df = df.groupby('NPP').sum()
aggs_dict = df['Aggs'].to_dict()
for npp in aggs_dict: aggs_dict[npp] = aggs_dict[npp].split()

# Build dataframe with aggregated NP-GPCRs conjugate to included NPPs
npp_list = list(pd.read_csv('Resources/30NPPs_byGa.csv')['Symbol'])
gene_order = npp_list
df = anno_counts[npp_list].copy()
for npp in npp_list: df[npp+'Rs'] = anno_counts[aggs_dict[npp]].sum(axis=1)
npr_list = [x+'Rs' for x in npp_list]
counts = df.copy()

# Update gene dictionary
#new_keys = [x+'Rs' for x in list(aggs_dict.keys())]
#new_values = list(aggs_dict.values())
#gene_dict.update(dict(zip(new_keys, new_values)))

# Build autocrine pair table

df = counts.copy()
npps = df[npp_list].copy()
nprs = df[npr_list].copy()

npps.columns = [s+'*' for s in npps.columns] 
nprs.columns = npps.columns
prods = npps * nprs

#bnpps = (npps+1-threshold).clip(0,1)
#bnprs = (nprs+1-threshold).clip(0,1)
#bprods = (prods+1-threshold).clip(0,1)

bnpps = (npps>=threshold).astype(int)
bnprs = (nprs>=threshold).astype(int)
bprods = (prods>=threshold).astype(int)

# Draw % autocrines by region heatmap

# Get a serial number and save a copy of this notebook
sn = pm.serial(5)
source_notebook_path = notebook_name+'.ipynb'
destination_notebook_path = 'Notebooks/'+notebook_name+'_'+sn+'.ipynb'
shutil.copy2(source_notebook_path, destination_notebook_path)

# Get the memory usage in percentage
print(f"Memory usage: {psutil.virtual_memory().percent}%")

In [ ]:
# Tally all NPPs
df = anno_counts.loc['01 Neuronal']
npp_tallies = df[npp_list].sum()
df = pd.DataFrame(npp_tallies)
df.to_csv(results_path+dataset+'_npp_tallies.csv')

In [ ]:
# Region stats by class
# Setup for Multi-mapper cell
major_list = ['00 All']+classes
major_label = 'class'
minor_label = 'region'
color_dict = region_colors
shorts_dict = region_shorts
zone_dict = region_zones
map_label = 'Region Stats for Class'


In [ ]:
# Subclass stats by region
# Setup for Multi-mapper cell
major_list = regions
major_label = 'region'
minor_label = 'subclass'
color_dict = subclass_colors
shorts_dict = subclass_shorts
zone_dict = subclass_zones
map_label = 'Subclass Stats for Region'


In [ ]:
# Cluster stats by subclass
# Setup for Multi-mapper cell
major_list = subclasses
major_label = 'subclass'
minor_label = 'cluster'
color_dict = cluster_colors
shorts_dict = None
zone_dict = cluster_zones
map_label = 'Cluster Stats for Subclass'


In [ ]:
# Multi-mapper
# Map %NPco's after setup, write results to a PDF and a ZIP archive of CSVs

row_stat = 'max' # 'max' or 'mean'
row_norm = True
row_sort = False
row_sort_all = False
log_floor_coef = 0.01
handle = dataset+'_'+notebook_name+'_'+row_stat+'_th'+str(threshold)+'_'

if row_sort: handle += 'sort_'
csv_file = csv_path+handle+minor_label+'_'+sn+'.csv'
zip_file = zip_path+handle+minor_label+'_'+sn+'.zip'
temp_csv = temp_path+'temp.csv'
pdf = PdfPages(results_path+'PDFs/'+handle+minor_label+'_'+sn+'.pdf')

sns.set_theme(font_scale=1.1, rc={'patch.linewidth': 0})
types = ['01 Neuronal']
#types = ['02 Glial']

zip_mode = 'w'
idx = pd.IndexSlice
for major in major_list:
    if major_label == 'class':
        if major == '00 All': gate = types,regions,classes,subclasses
        else: gate = types,regions,major,subclasses
    if major_label == 'region': gate = types,major,classes,subclasses
    if major_label == 'subclass': gate = types,regions,classes,major
    for i, metric in enumerate(['NPP','%ofAll','%NPP+','%ofNPP+','NPR','%NPR+','%ofNPR+']):
        units = ' %' if '%' in metric else ''
        log = False if '%' in metric else True        
        try: df = npps.loc[idx[gate], :]
        except KeyError: continue
        metric_label = map_label+' '+major+'\n\n'
        if metric == 'NPP':
            df = npps.loc[idx[gate], :]
            metric_label += 'Mean NPP Counts'
        if metric == '%NPP+':
            df = bnpps.loc[idx[gate], :]
            metric_label += 'Percent of all cells NPP+'
        if metric == 'NPR':
            df = nprs.loc[idx[gate], :]
            metric_label += 'Mean NP-GPCR Counts'
        if metric == '%NPR+':
            df = bnprs.loc[idx[gate], :]
            metric_label += 'Percent of all cells NP-GPCR+'
        if metric == '%ofAll': # Note: NaN values are operative here
            df = bprods.loc[idx[gate], :]
            metric_label += 'Percent of all cells NPP+ & NP-GPCR+'
        if metric == '%ofNPP+': # Note: NaN values are operative here
            df = bprods.loc[idx[gate], :]
            df = df / bnpps.loc[idx[gate], :]
            metric_label += 'Percent of NPP+ cells also NP-GPCR+'
        if metric == '%ofNPR+': # Note: NaN values are operative here
            df = bprods.loc[idx[gate], :]
            df = df / bnprs.loc[idx[gate], :]
            metric_label += 'Percent of NP-GPCR+ cells also NPP+'
        df = df.replace([np.inf, -np.inf], np.nan)
        df = df.groupby(minor_label).filter(lambda x: len(x) >= min_tally) # filter out small groups
        if len(df) == 0: continue
        grouped = df.groupby(minor_label)
        if '%' in metric:
            df = 100 * grouped.mean()
            units = ' %'
            cbar_label = '% Co-Expressing' if '%of' in metric else '% Expressing' 
        else:
            df = grouped.mean()
            units = ''
            cbar_label = 'Mean Counts'
        if i == 0 or row_sort_all:
            if row_sort: 
                if row_stat == 'max': gene_order = list(df[df.max().sort_values(ascending=False).index].columns)
                if row_stat == 'mean': gene_order = list(df[df.mean().sort_values(ascending=False).index].columns)
            else: gene_order = list(df.columns)
            cell_tally = grouped.count().iloc[:,0]
            vlines = []; zone = []
            for i, minor in enumerate(df.index):
                if zone_dict[minor] != zone: vlines += [i]; zone = zone_dict[minor]
            vlines += [len(df)]
        df = df[gene_order].copy()
        if metric in ['NPP','%NPP+']: df.columns = [s for s in [s.split('*')[0] for s in list(df.columns)]]
        if metric in ['NPR','%NPR+']: df.columns = [s+'*R' for s in [s.split('*')[0] for s in list(df.columns)]]
        if '%of' in metric: df.columns = [s+' & *Rs' for s in [s.split('*')[0] for s in list(df.columns)]]
        X = df.T
        colors = [color_dict[s] for s in list(X.columns)]
        if shorts_dict != None: X.columns = [shorts_dict[s] for s in list(X.columns)]
        # Plot the heatmap
        title = handle.replace('_',' ')+sn+'\n'+metric_label
        pm.draw_heatmap(X,cell_tally, colors, title, pdf, row_norm=row_norm, row_sort=False,
                                    row_stat = row_stat, vlines=vlines, units=units, log=log,
                                    log_floor_coef=log_floor_coef, cbar=True, cbar_label=cbar_label)
        # Append a cell tally row to X and start the CSV zip archive
        X.loc['cells'] = list(cell_tally)
        X.to_csv(temp_csv)
        with zipfile.ZipFile(zip_file, zip_mode) as zipf:
            arc_name = handle+major_label+major.split()[0]+'_'+metric+'_'+sn+'.csv'
            zipf.write(temp_csv, arcname=arc_name)
        zip_mode = 'a'
        print(arc_name,'written to',zip_file)

pdf.close()